In [60]:
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from os.path import expanduser
import os.path 
import numpy as np
import webbrowser as wb

#TODO:
# create helper functions: data quality binning
#--make a single "schedule/metadata" table with the liftoff and start of takeoff points and quality, computation of takeoff roll length

home = expanduser("~")
filepath = home+'\OneDrive - DOT OST\BADA4_Reduced_Thrust_Sensor_Path_Noise_Comparison_main'
filename = 'takeoff_distance - no positions.csv'
df = pd.read_csv(os.path.join(filepath, filename), index_col=["FLIGHT_ID", "TIME_OFFSET"])

df_liftoff = pd.DataFrame(df[df.DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF.isna() == False])
df_liftoff.reset_index(level = 1, inplace = True)

df_rollstart = df[df.DISTANCE_FROM_RUNWAY_END_AT_DETECTED_START_OF_TAKEOFF.isna() == False]
df_rollstart.reset_index(level = 1, inplace = True)

In [61]:
df.columns

Index(['P64: Duration of Takeoff (Seconds)',
       'TIME_ON_GROUND_BEFORE_LIFTOFF_(SECONDS)',
       'P64: Duration of Taxi Out (Minutes)', 'DURATION',
       'SPEED_SOUND_START_EVENT', 'HEAD_WIND', 'MSL_ALT', 'AFE_ALT',
       'TAS_START_EVENT', 'P64: True Airspeed at Liftoff (knots)',
       'MACH_NUMBER_START_EVENT', 'GS_SEGMENT', 'TAS_SEGMENT',
       'MACH_NUMBER_SEGMENT', 'DRAG', 'LIFT', 'DISTANCE_START_EVENT',
       'DISTANCE_END_EVENT', 'FUELFLOW_START_EVENT', 'FUELFLOW_SEGMENT', 'N1',
       'THRUST_START_EVENT', 'THRUST_SEGMENT', 'LIFTOFF',
       'Takeoff Runway Starting Latitude', 'Takeoff Runway Starting Longitude',
       'STATED_SEGMENT_START_OF_TAKEOFF', 'DISTANCE_FROM_RUNWAY_END',
       'DISTANCE_FROM_RUNWAY_END_AT_DETECTED_START_OF_TAKEOFF',
       'DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF', 'FLIGHT_ID.1',
       'APT_AIRCRAFT_RUNWAY_STAGE',
       'SHARE_OF_REMAINING_RUNWAY_LENGTH_AT_DETECTED_AIRBORNE_POINT_IN_RUNWAY_VICINITY',
       'FEET_OF_REMAINING_RUNWAY

In [62]:
df.head()

P64: Duration of Takeoff (Seconds)  \
FLIGHT_ID TIME_OFFSET                                       
1001880   855                                      42.375   
          856                                      42.375   
          857                                      42.375   
          858                                      42.375   
          859                                      42.375   

                       TIME_ON_GROUND_BEFORE_LIFTOFF_(SECONDS)  \
FLIGHT_ID TIME_OFFSET                                            
1001880   855                                              900   
          856                                              900   
          857                                              900   
          858                                              900   
          859                                              900   

                       P64: Duration of Taxi Out (Minutes)  DURATION  \
FLIGHT_ID TIME_OFFSET                                                  
1001880   855                                         10.3         5   
          856                                         10.3         5   
          857                                         10.3         5   
          858                                         10.3         5   
          859                                         10.3         1   

                       SPEED_SOUND_START_EVENT  HEAD_WIND  MSL_ALT   AFE_ALT  \
FLIGHT_ID TIME_OFFSET                                                          
1001880   855                       645.224609        0.0   670.75 -2.444625   
          856                       645.224609        0.0   670.50 -2.443270   
          857                       645.224609        0.0   669.75 -2.441950   
          858                       645.224609        0.0   669.75 -2.440629   
          859                       645.224609        0.0   669.50 -2.470281   

                       TAS_START_EVENT  P64: True Airspeed at Liftoff (knots)  \
FLIGHT_ID TIME_OFFSET                                                           
1001880   855                      0.0                               157.3125   
          856                      0.0                               157.3125   
          857                      0.0                               157.3125   
          858                      0.0                               157.3125   
          859                      0.0                               157.3125   

                       ...  DISTANCE_FROM_POINT_ONE_FEET  \
FLIGHT_ID TIME_OFFSET  ...                                 
1001880   855          ...                           NaN   
          856          ...                           NaN   
          857          ...                           NaN   
          858          ...                           NaN   
          859          ...                           NaN   

                       GROUNDSPEED_AT_POINT_ONE_KNOTS  \
FLIGHT_ID TIME_OFFSET                                   
1001880   855                                     NaN   
          856                                     NaN   
          857                                     NaN   
          858                                     NaN   
          859                                     NaN   

                       WELL-BEHAVED_TRAJECTORY  LATITUDE_AT_POINT_ONE  \
FLIGHT_ID TIME_OFFSET                                                   
1001880   855                              NaN                    NaN   
          856                              NaN                    NaN   
          857                              NaN                    NaN   
          858                              NaN                    NaN   
          859                              NaN                    NaN   

                       LONGITUDE_AT_POINT_ONE  \
FLIGHT_ID TIME_OFFSET                           
1001880   855                             NaN   
          856     

In [63]:
salient_rollstart_col_list = ['DISTANCE_FROM_THRESHOLD_AT_POINT_ONE', 'DISTANCE_FROM_POINT_ONE_FEET',
       'GROUNDSPEED_AT_POINT_ONE_KNOTS', 'WELL-BEHAVED_TRAJECTORY',
       'LATITUDE_AT_POINT_ONE', 'LONGITUDE_AT_POINT_ONE','STATED_SEGMENT_START_OF_TAKEOFF',
       'DISTANCE_FROM_RUNWAY_END_AT_DETECTED_START_OF_TAKEOFF']
extraneous_schedule_columns_list = [col for col in df_rollstart.columns if col not in salient_rollstart_col_list + ["DISTANCE_FROM_RUNWAY_END"]]



In [64]:
df_schedule = df_liftoff.drop(salient_rollstart_col_list, axis = 1).join(
    df_rollstart.drop(
        columns = extraneous_schedule_columns_list, axis = 1
    )
    , how = 'left'
    , lsuffix = '_liftoff' 
    , rsuffix = '_rollstart'
)

 len(df[df["WELL-BEHAVED_TRAJECTORY"] == 1]) rows in the sample are well-behaved

In [65]:
df_schedule["liftoff_detection_quality"] = abs(df_liftoff.DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF - df_liftoff.DISTANCE_FROM_RUNWAY_END) / df_liftoff.DISTANCE_FROM_RUNWAY_END 
#instantiate bins for grouping on the quality of liftoff detection
liftoff_detection_quality_bins = [0, 0.01] #, 0.05, 0.1, 1]
df_schedule["very_high_liftoff_detection_quality"] = df_schedule["liftoff_detection_quality"].between(liftoff_detection_quality_bins[0], liftoff_detection_quality_bins[1]) == True
flights_with_very_high_liftoff_detection_quality = df_schedule[df_schedule.very_high_liftoff_detection_quality == True].index.to_series()

In [67]:
df_high_qual_lift = df.join(flights_with_very_high_liftoff_detection_quality,how = 'inner')
fp = plot_metrics_for_individual_flights(df_high_qual_lift, -1)
wb.open(url=fp)

C:\Users\Lyle.Tripp\OneDrive - DOT OST\BADA4_Reduced_Thrust_Sensor_Path_Noise_Comparison_main\plotbook.html


True

In [83]:
df_schedule = df_schedule.loc[[1124373, 1128101],:]

In [84]:
x_max = stated_max = df_schedule.DISTANCE_FROM_RUNWAY_END_liftoff.max()
x_min = stated_min = df_schedule.DISTANCE_FROM_RUNWAY_END_liftoff.min()
y_max = detected_max = df_schedule.DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF.max()
y_min = detected_min = df_schedule.DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF.min()

perfect_fit_min = x_min if x_min < y_min else y_min
perfect_fit_max = x_max if x_max > y_max else y_max

fig = px.scatter(
    df_schedule.reset_index(level = 0)
    , title='Liftoff distance from runway end (feet)'
    , x='DISTANCE_FROM_RUNWAY_END_liftoff'
    ,y='DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF'
    , color="APT_AIRCRAFT_RUNWAY_STAGE.1" 
    , labels = {'DISTANCE_FROM_RUNWAY_END_liftoff':'As stated in CFDR',
                'DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF':'As detected from trajectory',
                'APT_AIRCRAFT_RUNWAY_STAGE.1': 'Airport / Aicraft Type / Runway / Stage Length ID'
               }
    , hover_data= ['FLIGHT_ID']#, 'DISTANCE_FROM_RUNWAY_END', 'DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF']
)

fig.add_trace(
    go.Scatter(
        x = [perfect_fit_min, perfect_fit_max]
        , y = [perfect_fit_min, perfect_fit_max]
        , mode = "lines"
        , name = 'Perfect Fit'
        , line = go.scatter.Line(color = 'black', dash = 'dash')
        , opacity=0.4
    )
)

fig.add_trace(
    go.Scatter(
        x = [perfect_fit_min, perfect_fit_max]
        , y = [perfect_fit_min+275, perfect_fit_max+275]
        , mode = "lines"
        , name = 'Perfect Fit + 275 ft'
        , line = go.scatter.Line(color = 'black', dash = 'longdash')
        , opacity=0.4
    )

)

fig.add_trace(
    go.Scatter(
        x = [perfect_fit_min, perfect_fit_max]
        , y = [perfect_fit_min+550, perfect_fit_max+550]
        , mode = "lines"
        , name = 'Perfect Fit + 550 ft'
        , line = go.scatter.Line(color = 'black', dash = 'longdashdot')
        , opacity=0.4
    )

)

fig.update_yaxes(
#     scaleanchor = "x",
    scaleratio = 1,
  )

fig.show()

file = fig.to_html()
with open(filepath+'\plot_liftoff.html', 'w') as f:
    f.write(file)
    

In [101]:
#two flights with similar stated liftoff point 
flights = [1128101, 1124373]
plot_metrics_for_individual_flights(df.loc[flights,])


C:\Users\Lyle.Tripp\OneDrive - DOT OST\BADA4_Reduced_Thrust_Sensor_Path_Noise_Comparison_main\plotbook_10.html


'C:\\Users\\Lyle.Tripp\\OneDrive - DOT OST\\BADA4_Reduced_Thrust_Sensor_Path_Noise_Comparison_main\\plotbook_10.html'

In [98]:
df.columns

Index(['P64: Duration of Takeoff (Seconds)',
       'TIME_ON_GROUND_BEFORE_LIFTOFF_(SECONDS)',
       'P64: Duration of Taxi Out (Minutes)', 'DURATION',
       'SPEED_SOUND_START_EVENT', 'HEAD_WIND', 'MSL_ALT', 'AFE_ALT',
       'TAS_START_EVENT', 'P64: True Airspeed at Liftoff (knots)',
       'MACH_NUMBER_START_EVENT', 'GS_SEGMENT', 'TAS_SEGMENT',
       'MACH_NUMBER_SEGMENT', 'DRAG', 'LIFT', 'DISTANCE_START_EVENT',
       'DISTANCE_END_EVENT', 'FUELFLOW_START_EVENT', 'FUELFLOW_SEGMENT', 'N1',
       'THRUST_START_EVENT', 'THRUST_SEGMENT', 'LIFTOFF',
       'Takeoff Runway Starting Latitude', 'Takeoff Runway Starting Longitude',
       'STATED_SEGMENT_START_OF_TAKEOFF', 'DISTANCE_FROM_RUNWAY_END',
       'DISTANCE_FROM_RUNWAY_END_AT_DETECTED_START_OF_TAKEOFF',
       'DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF', 'FLIGHT_ID.1',
       'APT_AIRCRAFT_RUNWAY_STAGE',
       'SHARE_OF_REMAINING_RUNWAY_LENGTH_AT_DETECTED_AIRBORNE_POINT_IN_RUNWAY_VICINITY',
       'FEET_OF_REMAINING_RUNWAY

In [ ]:
fig = px.scatter(
    df_rollstart
    , title='Start of takeoff roll, distance from runway end (feet)'
    , x='DISTANCE_FROM_RUNWAY_END'
    , y='DISTANCE_FROM_RUNWAY_END_AT_DETECTED_START_OF_TAKEOFF'
    , color="APT_AIRCRAFT_RUNWAY_STAGE" 
    , labels = {'DISTANCE_FROM_RUNWAY_END':'As stated in CFDR',
                'DISTANCE_FROM_RUNWAY_END_AT_DETECTED_START_OF_TAKEOFF':'As detected from trajectory',
                'APT_AIRCRAFT_RUNWAY_STAGE': 'Airport / Aicraft Type / Runway / Stage Length ID'
               }
)

fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )

fig.show()

file = fig.to_html()
with open(filepath+'\plot_rollstart.html', 'w') as f:
    f.write(file)

In [ ]:
fig_px = px.scatter(
    df_plot
    , title='Liftoff distance from runway end (feet)'
    , x='DISTANCE_FROM_RUNWAY_END'
    ,y='MSL_ALT'
    , color = "FLIGHT_ID"
    , labels = {'DISTANCE_FROM_RUNWAY_END':'Distance from runway end (feet)',
                'MSL_ALT':'Altitude above mean sea level (feet)',
                'APT_AIRCRAFT_RUNWAY_STAGE.1': 'Airport / Aicraft Type / Runway / Stage Length ID'
               }
)

file = fig_px.to_html()
with open(filepath+'\plot_high_quality_by_flight.html', 'w') as f:
    f.write(file)
    
fig_px.show()

In [74]:
plot_metrics_for_individual_flights(df, 2)

KeyError: 1001898

In [100]:
def plot_metrics_for_individual_flights(dataframe, flight_sample_size = 10):

    metric_maxes = {}

    plotbook_filename = 'plotbook' + '_' + str(flight_sample_size) if flight_sample_size != -1 else 'plotbook'
    df = dataframe
    if 'FLIGHT_ID' in df.columns:
        df_grouped_by_flight = df.drop(columns = ['FLIGHT_ID'], axis = 1).groupby("FLIGHT_ID")
        df["FLIGHT_ID"] = df["FLIGHT_ID"].astype(str)
    else:
        df_grouped_by_flight = df.groupby("FLIGHT_ID")
    
    fn = plotbook_filename + '.html'
    open( os.path.join(filepath,fn), 'w')
    #     #TODO: write plots to pdf file
    #     fn = plotbook_filename + '.pdf'

    flight_count = 0
    for flight, group in df_grouped_by_flight:
        flight_sample_size = len(group) if flight_sample_size == -1 else flight_sample_size

        if flight_count == flight_sample_size:
            break

        metrics = ['MSL_ALT', 'N1', 'TAS_SEGMENT']

        #create Figure with subplots for each metric for a given flight
        fig = make_subplots(rows=len(metrics), cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.02)

        for m in metrics:
            metric_maxes[m]=group[m].max() if m != 'N1' else 100

            #create and add a subplot to the Figure
            sub = go.Scatter(
                    x=group.DISTANCE_FROM_RUNWAY_END
                    , y=group[m]
                    , mode="markers"
                    , name = m
            )
            fig.add_trace(sub, row=metrics.index(m)+1, col=1)

    #         #turn off auto range adjustment
    #         fig.update_layout(
    #             yaxis_autorange = False
    #             )        
    #         fig.update_yaxes({}, row=metrics.index(m)+1, col=1)

            dl_cgtd = detected_liftoff_cumul_grnd_trk_dist = df_liftoff.loc[flight,"DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF"]
            fig.add_trace(
                go.Scatter(
                    x=[dl_cgtd, dl_cgtd]
                    , y=[0,metric_maxes[m]]
                    , mode = "lines"
                    , name = "Detected Liftoff"
                    , line = go.scatter.Line(color = 'gray')
                )
                , row=metrics.index(m)+1, col=1 
            )        

            rprt_liftoff_cgtd = reported_liftoff_cumul_grnd_trk_dist = df_liftoff.loc[flight,"DISTANCE_FROM_RUNWAY_END"]
            fig.add_trace(
                go.Scatter(
                    x=[rprt_liftoff_cgtd, rprt_liftoff_cgtd]
                    , y=[0,metric_maxes[m]]
                    , mode = "lines"
                    , name = "Reported Liftoff"
                    , line = go.scatter.Line(color = 'orange', dash = 'dash')
                )
                , row=metrics.index(m)+1, col=1 
            )

            fig.layout.yaxis.update(title_text = metrics[0])
            fig.layout.yaxis2.update(title_text = metrics[1], tickvals=[0] + np.arange(70,100,5))
            fig.layout.yaxis3.update(title_text = metrics[2])
            fig.layout.update(title_text = "Flight " + str(flight) + '<br>' + 'Airport / Aicraft Type / Runway / Stage Length ID: ' + 
                              df_liftoff.loc[flight,"APT_AIRCRAFT_RUNWAY_STAGE.1"])
        
        #write plots to html file         
        html = fig.to_html()
        with open( os.path.join(filepath,fn), 'a') as f:
            f.write(html) 

        flight_count = flight_count + 1
    print(os.path.join(filepath,fn))
        
    return (os.path.join(filepath,fn))